In [1]:
import pandas as pd

In [2]:
from src.GWY_news_utils import aggregate_content,calculate_attitude_score,load_attitude_dictionary,regression_x_y

In [3]:
from statsmodels.iolib.summary2 import summary_col
import statsmodels.api as sm

In [4]:

def ols_same_x(y_list,x):
    
    result_list = []
    
    for y in y_list:
        
        X,Y = regression_x_y(y,x,y_predict_lag=1,y_include_lag=0,x_include_lag=0)
        model = sm.OLS(Y,X)
        result = model.fit()
        result_list.append(result)
    
    return result_list

In [5]:
def ols_same_y(y,x_list):
    
    result_list = []
    
    check = 0
    
    for x in x_list:
        
        if check == 0:
            X,Y = regression_x_y(y,x,y_predict_lag=1,y_include_lag=0,x_include_lag=0)
            model = sm.OLS(Y,X)
            result = model.fit()
            result_list.append(result)
            check = 1
            
        X,Y = regression_x_y(y,x,y_predict_lag=1,y_include_lag=1,x_include_lag=0)
        model = sm.OLS(Y,X)
        result = model.fit()
        result_list.append(result)
    
    return result_list

In [6]:
data_directory = "data/extra_index_return/"
df = pd.read_excel(data_directory+"index_category_selected_2.xls")
df["交易日期_TrdDt"] = pd.to_datetime(df["交易日期_TrdDt"])

In [7]:
df_wide = pd.pivot_table(df,values="收盘价()_ClPr",index=["交易日期_TrdDt"],columns=["指数代码_IdxCd"])

In [8]:
df_wide.iloc[:,0:5].to_csv(data_directory+"index_category_selected_2_wide.csv")

In [12]:
sum(df_wide.index > "2016-09-25")

1

In [15]:
df_wide[df_wide.index > "2016-08-27"]

指数代码_IdxCd,903,904,905,906,907,399903,399904,399905,399906,399907
交易日期_TrdDt,,,,,,,,,,
2016-08-29,3075.04,4208.15,6427.53,3854.44,5198.72,3075.04,4208.15,6427.53,NaN,NaN
2016-08-30,3083.55,4203.05,6425.75,3857.53,5195.03,3083.55,4203.05,6425.75,NaN,NaN
2016-08-31,3104.51,4208.87,6434.21,3871.89,5202.04,3104.51,4208.87,6434.21,NaN,NaN
2016-09-01,3079.92,4176.02,6381.48,3841.02,5160.34,3079.92,4176.02,6381.48,NaN,NaN
2016-09-02,3096.00,4181.89,6356.95,3846.72,5153.03,3096.00,4181.89,6356.95,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2020-09-25,4607.61,5005.87,6236.89,4843.15,5575.73,4607.61,5005.87,6236.89,NaN,NaN
2020-09-28,4632.19,4993.38,6178.47,4841.63,5543.42,4632.19,4993.38,6178.47,NaN,NaN
2020-09-29,4631.74,5025.31,6228.77,4859.07,5583.50,4631.74,5025.31,6228.77,NaN,NaN


## Data Preparation

### Stock Return Data

In [9]:
data_directory = "data/extra_index_return/"

In [10]:
df = pd.read_csv(data_directory+"index_category_selected_2_wide.csv")

In [11]:
df["交易日期_TrdDt"] = pd.to_datetime(df["交易日期_TrdDt"])
df.index = df["交易日期_TrdDt"]
df = df.iloc[:,1:]
index_df = df

In [12]:
index_m = index_df.asfreq("BM",method="ffill")
index_m = index_m.pct_change()

index_w = index_df.asfreq("W",method="ffill")
index_w = index_w.pct_change()

index_d = index_df.asfreq("B",method="ffill")
index_d = index_d.pct_change()

In [13]:
index_d

,903,904,905,906,907
交易日期_TrdDt,,,,,
2010-01-04,NaN,NaN,NaN,NaN,NaN
2010-01-05,0.008151,0.008148,0.010409,0.008680,0.009262
2010-01-06,-0.008915,-0.000526,-0.001580,-0.005159,-0.001046
2010-01-07,-0.020005,-0.019486,-0.020027,-0.019882,-0.019752
2010-01-08,0.001686,0.004236,0.012401,0.004838,0.008261
...,...,...,...,...,...
2020-10-05,0.000000,0.000000,0.000000,0.000000,0.000000
2020-10-06,0.000000,0.000000,0.000000,0.000000,0.000000
2020-10-07,0.000000,0.000000,0.000000,0.000000,0.000000


### Government Index

In [14]:
gov_index_d = pd.read_csv("data/output/index_refinement_agg_daily.csv")
gov_index_w = pd.read_csv("data/output/index_refinement_agg_weekly.csv")
gov_index_m = pd.read_csv("data/output/index_refinement_agg_monthly.csv")

In [15]:
gov_index_d.index = pd.to_datetime(gov_index_d["date"])
gov_index_w.index = pd.to_datetime(gov_index_w["date"])
gov_index_m.index = pd.to_datetime(gov_index_m["date"])

In [16]:
gov_index_d = gov_index_d.iloc[:,1:]
gov_index_w = gov_index_w.iloc[:,1:]
gov_index_m = gov_index_m.iloc[:,1:]

### Government Index with Control

In [63]:
gov_index_d_e = pd.read_csv("data/output/daily_score_df_extra.csv")
gov_index_w_e = pd.read_csv("data/output/weekly_score_df_extra.csv")
gov_index_m_e = pd.read_csv("data/output/monthly_score_df_extra.csv")

In [64]:
gov_index_d_e.index = pd.to_datetime(gov_index_d_e["date"])
gov_index_w_e.index = pd.to_datetime(gov_index_w_e["date"])
gov_index_m_e.index = pd.to_datetime(gov_index_m_e["date"])

In [65]:
gov_index_d_e = gov_index_d_e.iloc[:,1:]
gov_index_w_e = gov_index_w_e.iloc[:,1:]
gov_index_m_e = gov_index_m_e.iloc[:,1:]

## Regression Analysis

### Merge X and Y

In [17]:
merge_df_m = index_m.merge(gov_index_m,how="inner",right_index=True,left_index=True)
merge_df_w = index_w.merge(gov_index_w,how="inner",right_index=True,left_index=True)
merge_df_d = index_d.merge(gov_index_d,how="inner",right_index=True,left_index=True)

In [67]:
merge_df_m_e = index_m.merge(gov_index_m_e,how="inner",right_index=True,left_index=True)
merge_df_w_e = index_w.merge(gov_index_w_e,how="inner",right_index=True,left_index=True)
merge_df_d_e = index_d.merge(gov_index_d_e,how="inner",right_index=True,left_index=True)

### Set up the regression

#### MLS With No control

In [152]:
regression_df = merge_df_m

In [153]:
regression_df.columns

Index(['903', '904', '905', '906', '907', 'reform_innovation', 'finance',
       'intervention_spending', 'regulation', 'intervention_tax_cut',
       'intervention_support', 'reform', 'intervention_x', 'reform_market',
       'economy', 'intervention_y'],
      dtype='object')

In [154]:
x_columns = ["economy","finance","reform","intervention_x","regulation"]

In [155]:
#y_list_1 = [regression_df["16"],regression_df["10"],regression_df["9"]]
#y_list_2 = [regression_df["399330"],regression_df["399009"],
#            regression_df["399007"],regression_df["399010"],regression_df["399011"]]
y_list_3 = [regression_df["903"],regression_df["904"],regression_df["905"]]

x = regression_df.loc[:,x_columns]

In [156]:
result_list = ols_same_x(y_list_3,x)

In [157]:
result_table = summary_col(result_list,
                             stars=True,
                             float_format="%0.3f",
                             info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                                        'R2':lambda x: "{:.2f}".format(x.rsquared)})

In [158]:
print(result_table.as_latex())

\begin{table}
\caption{}
\begin{center}
\begin{tabular}{lccc}
\hline
                &   903   &   904   &   905    \\
\midrule
const           & -0.030  & -0.020  & -0.019   \\
                & (0.023) & (0.027) & (0.026)  \\
economy         & 0.021   & 0.018   & 0.016    \\
                & (0.028) & (0.032) & (0.032)  \\
finance         & 0.023   & 0.125   & 0.133*   \\
                & (0.068) & (0.080) & (0.078)  \\
reform          & -0.006  & -0.034  & -0.038   \\
                & (0.022) & (0.026) & (0.025)  \\
intervention\_x & 0.051   & 0.019   & 0.008    \\
                & (0.031) & (0.037) & (0.036)  \\
regulation      & -0.013  & 0.006   & 0.015    \\
                & (0.036) & (0.042) & (0.040)  \\
R-squared       & 0.046   & 0.061   & 0.057    \\
                & 0.137   & 0.151   & 0.148    \\
N               & 53      & 53      & 53       \\
R2              & 0.14    & 0.15    & 0.15     \\
\hline
\end{tabular}
\end{center}
\end{table}


In [45]:
regression_df.columns

Index(['903', '904', '905', '906', '907', 'reform_innovation', 'finance',
       'intervention_spending', 'regulation', 'intervention_tax_cut',
       'intervention_support', 'reform', 'intervention_x', 'reform_market',
       'economy', 'intervention_y'],
      dtype='object')

#### SLS

In [119]:
regression_df = merge_df_d

In [128]:
y_list = [regression_df["903"],regression_df["904"],regression_df["905"]]
x = regression_df.loc[:,"intervention"]

In [129]:
result_list = ols_same_x(y_list,x)

In [130]:
result_table = summary_col(result_list,
                             stars=True,
                             float_format="%0.3f",
                             info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                                        'R2':lambda x: "{:.2f}".format(x.rsquared)})

In [131]:
print(result_table.as_latex())

\begin{table}
\caption{}
\begin{center}
\begin{tabular}{lccc}
\hline
             &   903   &   904   &   905    \\
\midrule
const        & 0.000   & 0.000   & -0.000   \\
             & (0.000) & (0.000) & (0.000)  \\
intervention & -0.000  & 0.000   & 0.000    \\
             & (0.000) & (0.001) & (0.001)  \\
R-squared    & -0.001  & -0.001  & -0.000   \\
             & 0.000   & 0.000   & 0.001    \\
N            & 1154    & 1154    & 1154     \\
R2           & 0.00    & 0.00    & 0.00     \\
\hline
\end{tabular}
\end{center}
\end{table}


### MLS with control 

In [319]:
regression_df.columns

Index(['9', '10', '16', '300', '399007', '399009', '399010', '399011',
       '399300', '399330', 'economy', 'reform', 'regulation', 'intervention',
       'total_length', 'number_of_text', 'EPU', 'EPU_change'],
      dtype='object')

In [428]:
regression_df = merge_df_m_e

In [457]:
y = regression_df["399011"]
x_list =[regression_df.iloc[:,10:14],regression_df.iloc[:,10:16],regression_df.iloc[:,10:]]

In [458]:
result_list = ols_same_y(y,x_list)

In [459]:
result_table = summary_col(result_list,
                             stars=True,
                             float_format="%0.3f",
                             info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                                        'R2':lambda x: "{:.2f}".format(x.rsquared)},
                          regressor_order=["const","economy","intervention","reform","regulation",
                                           "total_length","number_of_text","EPU","EPU_change"])

In [460]:
print(result_table.as_latex())

\begin{table}
\caption{}
\begin{center}
\begin{tabular}{lcccc}
\hline
                 & 399011 I & 399011\_y I & 399011\_y II & 399011\_y III  \\
\midrule
const            & -0.017   & -0.017      & 0.032        & 0.016          \\
                 & (0.034)  & (0.034)     & (0.042)      & (0.046)        \\
economy          & 0.020    & 0.020       & 0.045        & 0.069          \\
                 & (0.043)  & (0.043)     & (0.044)      & (0.048)        \\
intervention     & 0.084    & 0.079       & 0.042        & 0.051          \\
                 & (0.055)  & (0.055)     & (0.058)      & (0.059)        \\
reform           & -0.009   & -0.008      & -0.010       & -0.016         \\
                 & (0.022)  & (0.022)     & (0.022)      & (0.023)        \\
regulation       & -0.028   & -0.029      & -0.063       & -0.047         \\
                 & (0.063)  & (0.063)     & (0.064)      & (0.066)        \\
total\_length    &          &             & -0.000       & -0.000         

In [266]:
extra = pd.read_csv("data/extra_predictor_d.csv")
extra["date"] = pd.to_datetime(extra.iloc[:,0])
extra = extra.set_index("date")
#extra_m = extra_m.iloc[:,1:]

In [267]:
extra.shape

(1142, 4)

In [268]:
merge_df_d_extra = merge_df_d.merge(extra,left_index= True,right_index=True)

In [269]:
merge_df_d_extra.columns

Index(['903', '904', '905', '906', '907', 'economy', 'reform', 'intervention',
       'reform_market', 'finance', 'intervention_spending', 'regulation',
       'intervention_tax_cut', 'intervention_support', 'reform_innovation',
       'total_length', 'number_of_article', 'EPU', 'EPU_change'],
      dtype='object')

In [270]:
x_col_1 = ["economy","finance","reform","regulation","intervention"]
x_col_2 = x_col_1 + ["total_length","number_of_article"]
x_col_3 = x_col_2 + ["EPU","EPU_change"]

In [281]:
y = merge_df_d_extra["905"]
x_list =[merge_df_d_extra[x_col_1],merge_df_d_extra[x_col_2],merge_df_d_extra[x_col_3]]

In [282]:
result_list = ols_same_y(y,x_list)

In [283]:
result_list = ols_same_y(y,x_list)

In [284]:
result_table = summary_col(result_list,
                             stars=True,
                             float_format="%0.3f",
                             info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                                        'R2':lambda x: "{:.2f}".format(x.rsquared)},
                          regressor_order=["const","economy","finance","reform","regulation","intervention",
                                           "total_length","number_of_article","EPU","EPU_change","EPU_rolling"])

In [285]:
print(result_table.as_latex())

\begin{table}
\caption{}
\begin{center}
\begin{tabular}{lcccc}
\hline
                    &  905 I   & 905\_y I & 905\_y II & 905\_y III  \\
\midrule
const               & 0.000    & 0.000    & -0.000    & -0.001      \\
                    & (0.001)  & (0.001)  & (0.001)   & (0.001)     \\
economy             & -0.001** & -0.001** & -0.001**  & -0.001**    \\
                    & (0.001)  & (0.001)  & (0.001)   & (0.001)     \\
finance             & 0.001    & 0.001    & 0.001     & 0.001       \\
                    & (0.001)  & (0.001)  & (0.001)   & (0.001)     \\
reform              & 0.001    & 0.001    & 0.001     & 0.001       \\
                    & (0.001)  & (0.001)  & (0.001)   & (0.001)     \\
regulation          & -0.000   & -0.000   & -0.000    & -0.000      \\
                    & (0.001)  & (0.001)  & (0.001)   & (0.001)     \\
intervention        & 0.000    & 0.000    & 0.000     & 0.000       \\
                    & (0.001)  & (0.001)  & (0.001)   & (0.001)     \